In [1]:
import json
from ga.module_function import *
from ga.module_graph import *
from ga.module_population import *
from ga.module_individual import *
from ga.module_memset import *
from ga.module_evole import *

def read_data(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

def run_ga(data_path,result_store, population_size, generations, mutation_rate, crossover_rate, run_time_def):
    data = read_data(data_path)
    nodes = data['nodes']
    servers = data['server']
    clients = data['client']
    edges = data['edges']
    adj_matrix = [[] for _ in range(len(nodes))]
    for edge in edges:
        adj_matrix[edge[0]].append(edge[1])
        adj_matrix[edge[1]].append(edge[0])
    graph = Graph(len(nodes), len(clients), len(servers), len(clients), clients, servers, adj_matrix)
    func = Function()
    scenario_list = data['scenario']
    memSet = MemSet()
    result = {}
    run_time = 1
    for scenario in scenario_list:
        sum_delay_topo = 0
        delay_update = scenario["delay"]
        for delay in delay_update:
            sum_delay_topo  = sum_delay_topo + delay[2]
        bandwidth_update = scenario["bandwidth"]
        loss_update = scenario["loss"]
        request = scenario["request"]

        graph.updateGraph(delay_update, loss_update, bandwidth_update)

        pop = Population()
        pop.generate_population(graph, func, population_size, len(request), request, memSet)
        evole = Evolutionary()
        solutions = evole.evolve(pop, func, graph, generations, population_size, mutation_rate, crossover_rate, 2, 300)

        memSet.addAllPath(solutions, request)
        # print(solutions)
        Pareto = []

        for indi in solutions:
            objective = []
            objective.append(indi.objectives[0]/sum_delay_topo)
            objective.append(1+ indi.objectives[1]/200)
            objective.append(indi.objectives[2])
            Pareto.append(objective)
        result[run_time] = Pareto
        run_time = run_time + 1
        if run_time == run_time_def + 1:
            break
    with open(result_store, 'w') as f:
        json.dump(result, f)

ModuleNotFoundError: No module named 'ga'

In [4]:
run_ga(r"data\oxford\2_server.json",r'result\gamr\oxford_2_server_2.json', 300, 100, 0.1, 0.8, 50)

In [5]:
run_ga(r"data\oxford\5_server.json",r'result\gamr\oxford_5_server_2.json', 300, 100, 0.1, 0.8, 50)

In [6]:
run_ga(r"data\oxford\15_server.json",r'result\gamr\oxford_15_server_2.json', 300, 100, 0.1, 0.8, 50)

KeyboardInterrupt: 

In [2]:
run_ga(r"data\chinanet\2_server.json",r'result\gamr\chinanet_2_server_2.json', 300, 100, 0.1, 0.8, 50)

In [7]:
run_ga(r"data\chinanet\5_server.json",r'result\gamr\chinanet_5_server_2.json', 200, 100, 0.1, 0.8, 50)

In [ ]:
run_ga(r"data\chinanet\15_server.json",r'result\gamr\chinanet_15_server_2.json', 200, 100, 0.1, 0.8, 50)